In [1]:
import requests

# Your OpenWeatherMap API key
api_key = '337586e7326dcb828d7a386379093040'

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return f"Error: Unable to fetch weather data (Status code: {response.status_code})"




{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 20.07,
  'feels_like': 19.8,
  'temp_min': 17.08,
  'temp_max': 21.75,
  'pressure': 1008,
  'humidity': 64},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 250},
 'clouds': {'all': 75},
 'dt': 1717506738,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1717472806,
  'sunset': 1717531879},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [3]:
# Example usage
city = "Kolkata"
weather_data = get_weather(city)

# Display the weather data
weather_data

{'coord': {'lon': 88.3697, 'lat': 22.5697},
 'weather': [{'id': 721,
   'main': 'Haze',
   'description': 'haze',
   'icon': '50n'}],
 'base': 'stations',
 'main': {'temp': 31.97,
  'feels_like': 38.97,
  'temp_min': 31.97,
  'temp_max': 31.97,
  'pressure': 1001,
  'humidity': 79},
 'visibility': 4000,
 'wind': {'speed': 5.14, 'deg': 170},
 'clouds': {'all': 40},
 'dt': 1717507228,
 'sys': {'type': 1,
  'id': 9114,
  'country': 'IN',
  'sunrise': 1717456890,
  'sunset': 1717505312},
 'timezone': 19800,
 'id': 1275004,
 'name': 'Kolkata',
 'cod': 200}

In [4]:
import random

def generate_intent_dataset(num_samples=2000):
    cities = ["Kolkata", "New York", "London", "Tokyo", "Sydney", "Paris", "Berlin", "Toronto", "Mumbai", "Shanghai"]
    info_requests = ["Current Weather", "Weather Forecast"]
    dataset = []

    for _ in range(num_samples):
        city = random.choice(cities)
        info_request = random.choice(info_requests)
        user_prompt = f"What is the {info_request.lower()} in {city}?"
        intent_identification_response = {
            "City": city,
            "InfoRequest": info_request
        }
        dataset.append({
            "user_prompt": user_prompt,
            "intent_identification_response": intent_identification_response
        })

    return dataset

# Generate the dataset
intent_dataset = generate_intent_dataset()


In [6]:
# intent_dataset

In [9]:
import requests

# Your OpenWeatherMap API key
api_key = '337586e7326dcb828d7a386379093040'

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

def generate_response_dataset(intent_dataset):
    response_dataset = []

    for entry in intent_dataset:
        city = entry["intent_identification_response"]["City"]
        user_prompt = entry["user_prompt"]
        api_response = get_weather(city)
        
        if api_response is not None:
            weather = api_response['weather'][0]['description']
            temp = api_response['main']['temp']
            feels_like = api_response['main']['feels_like']
            humidity = api_response['main']['humidity']
            wind_speed = api_response['wind']['speed']
            clouds = api_response['clouds']['all']

            generated_response = (
                f"The weather in {city} is currently {weather} with a temperature of {temp}°C, "
                f"feeling like {feels_like}°C. The humidity is {humidity}%, and the wind is blowing at "
                f"{wind_speed} m/s. There are some clouds covering {clouds}% of the sky."
            )
            
            response_dataset.append({
                "user_prompt": user_prompt,
                "intent_identification_response": entry["intent_identification_response"],
                "api_response": api_response,
                "generated_response": generated_response
            })

    return response_dataset

# Generate the response dataset
response_dataset = generate_response_dataset(intent_dataset)


In [ ]:
response_dataset

In [10]:
import json

# Save the intent identification dataset
with open('intent_identification_dataset.json', 'w') as f:
    json.dump(intent_dataset, f, indent=4)

# Save the response generation dataset
with open('response_generation_dataset.json', 'w') as f:
    json.dump(response_dataset, f, indent=4)


In [2]:
import random
import requests

# Your OpenWeatherMap API key
api_key = '337586e7326dcb828d7a386379093040'

# City list and request types
cities = ["Kolkata", "New York", "London", "Tokyo", "Sydney", "Paris", "Berlin", "Toronto", "Mumbai", "Shanghai"]
info_requests = ["current weather", "weather forecast"]  # Lowercase for matching

# Intent classification and entity extraction
def extract_intent_and_entities(user_prompt):
    for intent in info_requests:
        if intent in user_prompt.lower():
            return {
                "intent": intent.replace(" ", "_"),
                "entities": {"city": user_prompt.split(" in ")[-1].strip()}
            }
    return None  # Return None if no intent is found

# Get weather data from API
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Generate the dataset
def generate_dataset(num_samples=10):
    dataset = []

    for _ in range(num_samples):
        city = random.choice(cities)
        info_request = random.choice(info_requests)
        user_prompt = f"What is the {info_request} in {city}?"

        intent_extraction = extract_intent_and_entities(user_prompt)
        if intent_extraction is None:
            continue  # Skip if no intent is recognized

        api_response = get_weather(city)
        if api_response is None:
            continue  # Skip if API call fails

        # Create the assistant's response
        if intent_extraction["intent"] == "current_weather":
            assistant_response = (
                f"The weather in {city} is currently {api_response['weather'][0]['description']} "
                f"with a temperature of {api_response['main']['temp']}°C "
                f"and wind speed of {api_response['wind']['speed']} m/s."
            )
        else:
            # Forecast response (implementation left as an exercise)
            assistant_response = f"This is a placeholder for the forecast in {city}."  

        dataset.append({
            "user_input": user_prompt,
            "intent_extraction": intent_extraction,
            "api_response": api_response,
            "assistant_response": assistant_response
        })

    return dataset

# Generate the dataset and print a sample
dataset = generate_dataset()
# print(dataset)  


In [4]:
import random
import requests

# Your OpenWeatherMap API key
api_key = '337586e7326dcb828d7a386379093040'

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def generate_dataset(num_samples=10):
    cities = ["Kolkata", "New York", "London", "Tokyo", "Sydney", "Paris", "Berlin", "Toronto", "Mumbai", "Shanghai"]
    info_requests = ["current_weather", "forecast_weather"]
    dates = ["today", "tomorrow"]  # Simplified for the example
    dataset = []

    for _ in range(num_samples):
        city = random.choice(cities)
        info_request = random.choice(info_requests)
        date = random.choice(dates) if info_request == "forecast_weather" else "today"
        user_prompt = f"What is the {info_request.replace('_', ' ')} in {city} {date}?"

        intent_extraction = {
            "intent": info_request,
            "entities": {
                "city": city,
                "date": date
            }
        }

        api_response = get_weather(city)
        
        if api_response is not None:
            weather = api_response['weather'][0]['description']
            temp = api_response['main']['temp']
            wind_speed = api_response['wind']['speed']
            clouds = api_response['clouds']['all']

            if info_request == "current_weather":
                generated_response = (
                    f"The weather in {city} is currently {weather} with a temperature of {temp}°C and "
                    f"wind speed of {wind_speed} meters per second."
                )
            else:
                forecast = [
                    {"date": date, "condition": weather, "high": temp, "low": temp - 5}  # Simplified for example
                ]
                generated_response = (
                    f"The forecast for {city} {date} is {weather} with a high of {temp} degrees Celsius and a low of {temp - 5} degrees Celsius."
                )

            dataset.append({
                "user_input": user_prompt,
                "intent_extraction": intent_extraction,
                "api_response": {
                    "location": f"{city}, {api_response['sys']['country']}",
                    "temperature": temp,
                    "description": weather,
                    "wind_speed": wind_speed,
                    "clouds": clouds,
                    "forecast": forecast if info_request == "forecast_weather" else None
                },
                "assistant_response": generated_response
            })

    return dataset

# Generate the dataset
final_dataset = generate_dataset()

# Print some examples
for example in final_dataset[:5]:
    print(example)


{'user_input': 'What is the current weather in Mumbai today?', 'intent_extraction': {'intent': 'current_weather', 'entities': {'city': 'Mumbai', 'date': 'today'}}, 'api_response': {'location': 'Mumbai, IN', 'temperature': 30.99, 'description': 'mist', 'wind_speed': 4.12, 'clouds': 40, 'forecast': None}, 'assistant_response': 'The weather in Mumbai is currently mist with a temperature of 30.99°C and wind speed of 4.12 meters per second.'}
{'user_input': 'What is the forecast weather in Kolkata today?', 'intent_extraction': {'intent': 'forecast_weather', 'entities': {'city': 'Kolkata', 'date': 'today'}}, 'api_response': {'location': 'Kolkata, IN', 'temperature': 31.97, 'description': 'haze', 'wind_speed': 1.54, 'clouds': 40, 'forecast': [{'date': 'today', 'condition': 'haze', 'high': 31.97, 'low': 26.97}]}, 'assistant_response': 'The forecast for Kolkata today is haze with a high of 31.97 degrees Celsius and a low of 26.97 degrees Celsius.'}
{'user_input': 'What is the current weather in

In [5]:
final_dataset

[{'user_input': 'What is the current weather in Mumbai today?',
  'intent_extraction': {'intent': 'current_weather',
   'entities': {'city': 'Mumbai', 'date': 'today'}},
  'api_response': {'location': 'Mumbai, IN',
   'temperature': 30.99,
   'description': 'mist',
   'wind_speed': 4.12,
   'clouds': 40,
   'forecast': None},
  'assistant_response': 'The weather in Mumbai is currently mist with a temperature of 30.99°C and wind speed of 4.12 meters per second.'},
 {'user_input': 'What is the forecast weather in Kolkata today?',
  'intent_extraction': {'intent': 'forecast_weather',
   'entities': {'city': 'Kolkata', 'date': 'today'}},
  'api_response': {'location': 'Kolkata, IN',
   'temperature': 31.97,
   'description': 'haze',
   'wind_speed': 1.54,
   'clouds': 40,
   'forecast': [{'date': 'today',
     'condition': 'haze',
     'high': 31.97,
     'low': 26.97}]},
  'assistant_response': 'The forecast for Kolkata today is haze with a high of 31.97 degrees Celsius and a low of 26.97

In [6]:
import random
import requests

# Your OpenWeatherMap API key
api_key = '337586e7326dcb828d7a386379093040'

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_forecast(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def generate_dataset(num_samples=10):
    cities = ["Kolkata", "New York", "London", "Tokyo", "Sydney", "Paris", "Berlin", "Toronto", "Mumbai", "Shanghai"]
    info_requests = ["current_weather", "forecast_weather"]
    dates = ["today", "tomorrow"]  # Simplified for the example
    dataset = []

    for _ in range(num_samples):
        city = random.choice(cities)
        info_request = random.choice(info_requests)
        date = random.choice(dates) if info_request == "forecast_weather" else "today"
        user_prompt = f"What is the {info_request.replace('_', ' ')} in {city} {date}?"

        intent_extraction = {
            "intent": info_request,
            "entities": {
                "city": city,
                "date": date
            }
        }

        if info_request == "current_weather":
            api_response = get_weather(city)
        else:
            api_response = get_forecast(city)
        
        if api_response is not None:
            if info_request == "current_weather":
                weather = api_response['weather'][0]['description']
                temp = api_response['main']['temp']
                wind_speed = api_response['wind']['speed']
                clouds = api_response['clouds']['all']

                generated_response = (
                    f"The weather in {city} is currently {weather} with a temperature of {temp}°C and "
                    f"wind speed of {wind_speed} meters per second."
                )
            else:
                forecasts = api_response['list']
                random_forecast = random.choice(forecasts)
                forecast_temp = random_forecast['main']['temp']
                forecast_weather = random_forecast['weather'][0]['description']

                generated_response = (
                    f"The forecast for {city} {date} is {forecast_weather} with a temperature of {forecast_temp}°C."
                )

            dataset.append({
                "user_input": user_prompt,
                "intent_extraction": intent_extraction,
                "api_response": api_response,
                "assistant_response": generated_response
            })

    return dataset

# Generate the dataset
final_dataset = generate_dataset()

# Print some examples
for example in final_dataset[:5]:
    print(example)


{'user_input': 'What is the current weather in Paris today?', 'intent_extraction': {'intent': 'current_weather', 'entities': {'city': 'Paris', 'date': 'today'}}, 'api_response': {'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 21.66, 'feels_like': 21, 'temp_min': 20.14, 'temp_max': 22.77, 'pressure': 1017, 'humidity': 43}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 330}, 'clouds': {'all': 0}, 'dt': 1717783369, 'sys': {'type': 2, 'id': 2041230, 'country': 'FR', 'sunrise': 1717732104, 'sunset': 1717789856}, 'timezone': 7200, 'id': 2988507, 'name': 'Paris', 'cod': 200}, 'assistant_response': 'The weather in Paris is currently clear sky with a temperature of 21.66°C and wind speed of 3.09 meters per second.'}
{'user_input': 'What is the forecast weather in Sydney today?', 'intent_extraction': {'intent': 'forecast_weather', 'entities': {'city': 'Sydney', 'date': 'tod

In [10]:
import random
import requests
from datetime import datetime, timedelta

# Your OpenWeatherMap API key
api_key = '337586e7326dcb828d7a386379093040'

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_forecast(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def generate_random_future_date():
    # Generate a random number of days between 1 and 5
    days = random.randint(1, 5)
    future_date = datetime.now() + timedelta(days=days)
    return future_date.strftime('%Y-%m-%d')

def generate_dataset(num_samples=20):
    cities = [
        "Kolkata", "New York", "London", "Tokyo", "Sydney", "Paris", "Berlin", "Toronto",
        "Mumbai", "Shanghai", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia",
        "San Antonio", "San Diego", "Dallas", "San Jose", "Austin"
    ]
    info_requests = ["current_weather", "forecast_weather"]
    dates = ["today", "tomorrow"] + [generate_random_future_date() for _ in range(8)]
    dataset = []

    for _ in range(num_samples):
        city = random.choice(cities)
        info_request = random.choice(info_requests)
        date = random.choice(dates) if info_request == "forecast_weather" else "today"
        user_prompt = f"What is the {info_request.replace('_', ' ')} in {city} {date}?"

        intent_extraction = {
            "intent": info_request,
            "entities": {
                "city": city,
                "date": date
            }
        }

        if info_request == "current_weather":
            api_response = get_weather(city)
        else:
            api_response = get_forecast(city)
        
        if api_response is not None:
            if info_request == "current_weather":
                weather = api_response['weather'][0]['description']
                temp = api_response['main']['temp']
                wind_speed = api_response['wind']['speed']
                clouds = api_response['clouds']['all']

                generated_response = (
                    f"The weather in {city} is currently {weather} with a temperature of {temp}°C and "
                    f"wind speed of {wind_speed} meters per second."
                )

                dataset.append({
                    "user_input": user_prompt,
                    "intent_extraction": intent_extraction,
                    "api_response": api_response,
                    "assistant_response": generated_response
                })
            
            elif info_request == "forecast_weather":
                forecast_list = api_response['list']
                filtered_forecasts = [forecast for forecast in forecast_list if date in forecast['dt_txt']]
                if filtered_forecasts:
                    selected_forecast = filtered_forecasts[0]
                    forecast_temp = selected_forecast['main']['temp']
                    forecast_weather = selected_forecast['weather'][0]['description']

                    generated_response = (
                        f"The forecast for {city} on {date} is {forecast_weather} with a temperature of {forecast_temp}°C."
                    )

                    dataset.append({
                        "user_input": user_prompt,
                        "intent_extraction": intent_extraction,
                        "api_response": api_response,
                        "assistant_response": generated_response
                    })

    return dataset

# Generate the dataset
final_dataset = generate_dataset()

# Print some examples
for example in final_dataset[:5]:
    print(example)


{'user_input': 'What is the current weather in Toronto today?', 'intent_extraction': {'intent': 'current_weather', 'entities': {'city': 'Toronto', 'date': 'today'}}, 'api_response': {'coord': {'lon': -79.4163, 'lat': 43.7001}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 16.33, 'feels_like': 16.08, 'temp_min': 14.97, 'temp_max': 18.62, 'pressure': 1001, 'humidity': 79}, 'visibility': 10000, 'wind': {'speed': 11.32, 'deg': 270, 'gust': 14.4}, 'clouds': {'all': 75}, 'dt': 1717784543, 'sys': {'type': 2, 'id': 2043365, 'country': 'CA', 'sunrise': 1717752989, 'sunset': 1717808223}, 'timezone': -14400, 'id': 6167865, 'name': 'Toronto', 'cod': 200}, 'assistant_response': 'The weather in Toronto is currently broken clouds with a temperature of 16.33°C and wind speed of 11.32 meters per second.'}
{'user_input': 'What is the current weather in London today?', 'intent_extraction': {'intent': 'current_weather', 'ent

In [11]:
final_dataset[0]

{'user_input': 'What is the current weather in Toronto today?',
 'intent_extraction': {'intent': 'current_weather',
  'entities': {'city': 'Toronto', 'date': 'today'}},
 'api_response': {'coord': {'lon': -79.4163, 'lat': 43.7001},
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}],
  'base': 'stations',
  'main': {'temp': 16.33,
   'feels_like': 16.08,
   'temp_min': 14.97,
   'temp_max': 18.62,
   'pressure': 1001,
   'humidity': 79},
  'visibility': 10000,
  'wind': {'speed': 11.32, 'deg': 270, 'gust': 14.4},
  'clouds': {'all': 75},
  'dt': 1717784543,
  'sys': {'type': 2,
   'id': 2043365,
   'country': 'CA',
   'sunrise': 1717752989,
   'sunset': 1717808223},
  'timezone': -14400,
  'id': 6167865,
  'name': 'Toronto',
  'cod': 200},
 'assistant_response': 'The weather in Toronto is currently broken clouds with a temperature of 16.33°C and wind speed of 11.32 meters per second.'}